In [1]:
!pip install plotly

import plotly.express as px
import plotly.graph_objects as go

# Optional: For JupyterLab users, might need this to enable Plotly support
# !jupyter labextension install jupyterlab-plotly

# To render plots inline:
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

import pandas as pd

# Adjust your path as needed
csv_path = "/Users/jamesjackson/Documents/liverpool_crime_analysis/csv_files/clean_data_final.csv"
df = pd.read_csv(csv_path, parse_dates=['month'])

!pip install ipywidgets
import ipywidgets as widgets
from IPython.display import display

dropdown = widgets.Dropdown(
    options=['Option 1', 'Option 2'],
    description='Select:',
)
display(dropdown)

Dropdown(description='Select:', options=('Option 1', 'Option 2'), value='Option 1')

In [2]:
import plotly.graph_objects as go
import pandas as pd

# Load data
csv_path = "/Users/jamesjackson/Documents/liverpool_crime_analysis/csv_files/clean_data_final.csv"
df = pd.read_csv(csv_path, parse_dates=['month'])

df['month_str'] = df['month'].dt.strftime('%b %Y')

# Map crime_type to crime_group
group_mapping = {
    'Burglary': 'Theft/Burglary/Robbery',
    'Other theft': 'Theft/Burglary/Robbery',
    'Vehicle crime': 'Theft/Burglary/Robbery',
    'Shoplifting': 'Theft/Burglary/Robbery',
    'Robbery': 'Theft/Burglary/Robbery',
    'Theft from the person': 'Theft/Burglary/Robbery',

    'Violence and sexual offences': 'Violent/Sexual',
    'Public order': 'Violent/Sexual',

    'Drugs': 'Drugs',
    'Possession of weapons': 'Drugs',

    'Criminal damage and arson': 'Property Damage',
    'Bicycle theft': 'Theft/Burglary/Robbery',
    'Other crime': 'Other',
    'Anti-social behaviour': 'ASB'
}
df['crime_group'] = df['crime_type'].map(group_mapping)

# Define your top 9 locations to filter for
top_locations = [
    "On or near Parking Area",
    "On or near Supermarket",
    "On or near Shopping Area",
    "On or near Petrol Station",
    "On or near Nightclub",
    "On or near Sports/Recreation Area",
    "On or near Further/Higher Educational Building",
    "On or near Hospital",
    "On or near Police Station"
]

# Create 'location_group' column: top locations or 'Other'
df['location_group'] = df['location'].apply(lambda x: x if x in top_locations else 'Other')

# Extract region from lsoa_name (assumes region is first word before a space)
df['region'] = df['lsoa_name'].apply(lambda x: x.split()[0] if isinstance(x, str) else 'Unknown')

# Find top 5 regions by crime count
top_regions = df['region'].value_counts().nlargest(5).index.tolist()

# Create 'region_group' column: top regions or 'Other'
df['region_group'] = df['region'].apply(lambda x: x if x in top_regions else 'Other')


# Define outcome groups explicitly
positive_outcomes = [
    "Action to be taken by another organisation",
    "Offender given a drugs possession warning",
    "Suspect charged as part of another case",
    "Offender given a caution",
    "Local resolution"
]
neutral_outcomes = [
    "Status update unavailable",
    "Under investigation",
    "Awaiting court outcome",
    "Court result unavailable"
]
negative_outcomes = [
    "Unable to prosecute suspect",
    "Investigation complete; no suspect identified",
    "Further action is not in the public interest",
    "Further investigation is not in the public interest",
    "Formal action is not in the public interest"
]

# Map outcome_group
def map_outcome_group(outcome):
    if outcome in positive_outcomes:
        return 'Positive'
    elif outcome in neutral_outcomes:
        return 'Neutral'
    elif outcome in negative_outcomes:
        return 'Negative'
    else:
        return None

df['outcome_group'] = df['last_outcome_category'].apply(map_outcome_group)

# Drop rows without outcome_group
df_filtered = df.copy()

# Dropdown options
crime_groups = sorted(df_filtered['crime_group'].dropna().unique())
dropdown_options = ['All'] + crime_groups

# Define pastel colors for outcome groups
color_map = {
    'Positive': '#A8D5BA',  # Soft Green
    'Neutral': '#C0C0C0',   # Light Grey
    'Negative': '#F7B7A3'   # Soft Coral
}

# Create dropdown widget
dropdown = widgets.Dropdown(
    options=dropdown_options,
    description='Crime Group:',
    value='All'
)

month_options = ['All'] + sorted(df['month_str'].unique())
month_dropdown = widgets.Dropdown(
    options=month_options,
    description='Month:',
    value='All'
)

location_options = ['All'] + sorted(df['location_group'].unique())

location_dropdown = widgets.Dropdown(
    options=location_options,
    description='Location:',
    value='All'
)

region_options = ['All'] + sorted(df['region_group'].unique())

region_dropdown = widgets.Dropdown(
    options=region_options,
    description='Region:',
    value='All'
)

outcome_groups = ['All', 'Positive', 'Neutral', 'Negative']

outcome_dropdown = widgets.Dropdown(
    options=outcome_groups,
    description='Outcome Group:',
    value='All'
)

dropdown_crime_group = dropdown
dropdown_month = month_dropdown
dropdown_location = location_dropdown
dropdown_region = region_dropdown

# Calculate total crimes KPI
def update_kpi(selected_group, selected_month, selected_location, selected_region, selected_outcome):
    data = df_filtered.copy()

    if selected_group != 'All':
        data = data[data['crime_group'] == selected_group]

    if selected_month != 'All':
        data = data[data['month_str'] == selected_month]

    if selected_location != 'All':
        data = data[data['location_group'] == selected_location]

    if selected_region != 'All':
        data = data[data['region_group'] == selected_region]

    if selected_outcome != 'All':
        data = data[data['outcome_group'] == selected_outcome]
    
    total_crimes = len(data)

    fig = go.Figure(go.Indicator(
        mode="number",
        value=total_crimes,
        title={"text": "Total Crimes Recorded"},
        number={'valueformat': ','}
    ))

    fig.update_layout(height=250)
    fig.show()


widgets.interact(update_kpi,
                 selected_group=dropdown_crime_group,
                 selected_month=dropdown_month,
                 selected_location=dropdown_location,
                 selected_region=dropdown_region,
                 selected_outcome=outcome_dropdown)

interactive(children=(Dropdown(description='Crime Group:', options=('All', 'ASB', 'Drugs', 'Other', 'Property …

<function __main__.update_kpi(selected_group, selected_month, selected_location, selected_region, selected_outcome)>

In [3]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Load data
csv_path = "/Users/jamesjackson/Documents/liverpool_crime_analysis/csv_files/clean_data_final.csv"
df = pd.read_csv(csv_path)

# Convert 'month' column to datetime and create a formatted string for dropdown
df['month'] = pd.to_datetime(df['month'])
df['month_str'] = df['month'].dt.strftime('%b %Y')

# Map crime_type to crime_group
group_mapping = {
    'Violence and sexual offences': 'Violent / Sexual',
    'Criminal damage and arson': 'Violent / Sexual',
    'Vehicle crime': 'Vehicle Crime',
    'Burglary': 'Theft / Burglary / Robbery',
    'Theft from the person': 'Theft / Burglary / Robbery',
    'Shoplifting': 'Theft / Burglary / Robbery',
    'Other theft': 'Theft / Burglary / Robbery',
    'Bicycle theft': 'Theft / Burglary / Robbery',
    'Drugs': 'Drug-Related Crime',
    'Anti-social behaviour': 'Anti-Social Behaviour',
    'Other crime': 'Other Crime',
    'Possession of weapons': 'Weapons Offences'
}
df['crime_group'] = df['crime_type'].map(group_mapping)

# Define your top 9 locations to filter for
top_locations = [
    "On or near Parking Area",
    "On or near Supermarket",
    "On or near Shopping Area",
    "On or near Petrol Station",
    "On or near Nightclub",
    "On or near Sports/Recreation Area",
    "On or near Further/Higher Educational Building",
    "On or near Hospital",
    "On or near Police Station"
]

# Create 'location_group' column: top locations or 'Other'
df['location_group'] = df['location'].apply(lambda x: x if x in top_locations else 'Other')

# Extract region from lsoa_name (assumes region is first word before a space)
df['region'] = df['lsoa_name'].apply(lambda x: x.split()[0] if isinstance(x, str) else 'Unknown')

# Find top 5 regions by crime count
top_regions = df['region'].value_counts().nlargest(5).index.tolist()

# Create 'region_group' column: top regions or 'Other'
df['region_group'] = df['region'].apply(lambda x: x if x in top_regions else 'Other')


# Define outcome groups explicitly
positive_outcomes = [
    "Action to be taken by another organisation",
    "Offender given a drugs possession warning",
    "Suspect charged as part of another case",
    "Offender given a caution",
    "Local resolution"
]
neutral_outcomes = [
    "Status update unavailable",
    "Under investigation",
    "Awaiting court outcome",
    "Court result unavailable"
]
negative_outcomes = [
    "Unable to prosecute suspect",
    "Investigation complete; no suspect identified",
    "Further action is not in the public interest",
    "Further investigation is not in the public interest",
    "Formal action is not in the public interest"
]

# Map outcome_group
def map_outcome_group(outcome):
    if outcome in positive_outcomes:
        return 'Positive'
    elif outcome in neutral_outcomes:
        return 'Neutral'
    elif outcome in negative_outcomes:
        return 'Negative'
    else:
        return None

df['outcome_group'] = df['last_outcome_category'].apply(map_outcome_group)

# Drop rows without outcome_group
df_filtered = df.dropna(subset=['outcome_group'])

# Dropdown options
crime_groups = sorted(df_filtered['crime_group'].dropna().unique())
dropdown_options = ['All'] + crime_groups

# Define pastel colors for outcome groups
color_map = {
    'Positive': '#A8D5BA',  # Soft Green
    'Neutral': '#C0C0C0',   # Light Grey
    'Negative': '#F7B7A3'   # Soft Coral
}

# Create dropdown widget
dropdown = widgets.Dropdown(
    options=dropdown_options,
    description='Crime Group:',
    value='All'
)

month_options = ['All'] + sorted(df['month_str'].unique())
month_dropdown = widgets.Dropdown(
    options=month_options,
    description='Month:',
    value='All'
)

location_options = ['All'] + sorted(df['location_group'].unique())

location_dropdown = widgets.Dropdown(
    options=location_options,
    description='Location:',
    value='All'
)

region_options = ['All'] + sorted(df['region_group'].unique())

region_dropdown = widgets.Dropdown(
    options=region_options,
    description='Region:',
    value='All'
)

# Function to update pie chart
def update_pie(selected_group, selected_month, selected_location, selected_region):
    data = df_filtered

    if selected_group != 'All':
        data = data[data['crime_group'] == selected_group]

    if selected_month != 'All':
        data = data[data['month'] == selected_month]

    if selected_location != 'All':
        data = data[data['location_group'] == selected_location]

    if selected_region != 'All':
        data = data[data['region_group'] == selected_region]

    outcome_percentages = data['outcome_group'].value_counts(normalize=True) * 100
    labels = outcome_percentages.index.tolist()
    values = outcome_percentages.values.tolist()
    colors = [color_map[label] for label in labels]

    fig = go.Figure(data=[go.Pie(
        labels=labels,
        values=values,
        hole=0.4,
        marker=dict(colors=colors, line=dict(color='black', width=1.5)),
        textinfo='percent',
        textfont=dict( color='black'),
        hovertemplate='%{label}<br>%{percent}<extra></extra>',
    )])

    fig.update_layout(
        width=800,   # width in pixels
        height=500,
        title=dict(
            text="Outcome Group Distribution",
            font=dict( color="#003366")
        ),
        
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.2,
            xanchor="center",
            x=0.5
        ),
        margin=dict(t=50, b=50)
    )

    fig.show()


# Link dropdown to update function
widgets.interact(update_pie,
                 selected_group=dropdown,
                 selected_month=month_dropdown,
                 selected_location=location_dropdown,
                 selected_region=region_dropdown)



interactive(children=(Dropdown(description='Crime Group:', options=('All', 'Drug-Related Crime', 'Other Crime'…

<function __main__.update_pie(selected_group, selected_month, selected_location, selected_region)>

In [4]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Load data
csv_path = "/Users/jamesjackson/Documents/liverpool_crime_analysis/csv_files/clean_data_final.csv"
df = pd.read_csv(csv_path)

# Convert 'month' column to datetime and create a formatted string for dropdown
df['month'] = pd.to_datetime(df['month'])
df['month_str'] = df['month'].dt.strftime('%b %Y')

# Map crime_type to crime_group
group_mapping = {
    'Violence and sexual offences': 'Violent/Sexual',
    'Criminal damage and arson': 'Violent/Sexual',
    'Possession of weapons': 'Weapons Offences',
    'Vehicle crime': 'Vehicle Crime',
    'Burglary': 'Theft/Burglary/Robbery',
    'Theft from the person': 'Theft/Burglary/Robbery',
    'Shoplifting': 'Theft/Burglary/Robbery',
    'Other theft': 'Theft/Burglary/Robbery',
    'Bicycle theft': 'Theft/Burglary/Robbery',
    'Drugs': 'Drug-Related Crime',
    'Anti-social behaviour': 'Anti-Social Behaviour',
    'Other crime': 'Other Crime'
}


df['crime_group'] = df['crime_type'].map(group_mapping)
df = df.dropna(subset=['crime_group'])  # drops any crime_type not mapped above


# Define your top 9 locations to filter for
top_locations = [
    "On or near Parking Area",
    "On or near Supermarket",
    "On or near Shopping Area",
    "On or near Petrol Station",
    "On or near Nightclub",
    "On or near Sports/Recreation Area",
    "On or near Further/Higher Educational Building",
    "On or near Hospital",
    "On or near Police Station"
]

# Create 'location_group' column: top locations or 'Other'
df['location_group'] = df['location'].apply(lambda x: x if x in top_locations else 'Other')

# Extract region from lsoa_name (assumes region is first word before a space)
df['region'] = df['lsoa_name'].apply(lambda x: x.split()[0] if isinstance(x, str) else 'Unknown')

# Find top 5 regions by crime count
top_regions = df['region'].value_counts().nlargest(5).index.tolist()

# Create 'region_group' column: top regions or 'Other'
df['region_group'] = df['region'].apply(lambda x: x if x in top_regions else 'Other')


# Define outcome groups explicitly
positive_outcomes = [
    "Action to be taken by another organisation",
    "Offender given a drugs possession warning",
    "Suspect charged as part of another case",
    "Offender given a caution",
    "Local resolution"
]
neutral_outcomes = [
    "Status update unavailable",
    "Under investigation",
    "Awaiting court outcome",
    "Court result unavailable"
]
negative_outcomes = [
    "Unable to prosecute suspect",
    "Investigation complete; no suspect identified",
    "Further action is not in the public interest",
    "Further investigation is not in the public interest",
    "Formal action is not in the public interest"
]

# Map outcome_group
def map_outcome_group(outcome):
    if outcome in positive_outcomes:
        return 'Positive'
    elif outcome in neutral_outcomes:
        return 'Neutral'
    elif outcome in negative_outcomes:
        return 'Negative'
    else:
        return None

df['outcome_group'] = df['last_outcome_category'].apply(map_outcome_group)

# Drop rows without outcome_group
# df_filtered = df.dropna(subset=['outcome_group'])
df_filtered = df.copy()


# Dropdown options
df['crime_group'] = df['crime_type'].map(group_mapping)
df['crime_group'] = df['crime_group'].fillna('Other Crime')

# Define pastel colors for outcome groups
# Fix keys to match crime_group labels exactly
color_map = {
    'Violent/Sexual': '#F7B7A3',
    'Theft/Burglary/Robbery': '#A8D5BA',
    'Anti-Social Behaviour': '#C0C0C0',
    'Drug-Related Crime': '#A3B4F7',
    'Vehicle Crime': '#F7E8A3',
    'Weapons Offences': '#D5A3F7',
    'Other Crime': '#F7C5A3'
}

# Create dropdown widget

month_options = ['All'] + sorted(df['month_str'].unique())
month_dropdown = widgets.Dropdown(
    options=month_options,
    description='Month:',
    value='All'
)

location_options = ['All'] + sorted(df['location_group'].unique())

location_dropdown = widgets.Dropdown(
    options=location_options,
    description='Location:',
    value='All'
)

region_options = ['All'] + sorted(df['region_group'].unique())

region_dropdown = widgets.Dropdown(
    options=region_options,
    description='Region:',
    value='All'
)

outcome_options = ['All'] + sorted(df_filtered['outcome_group'].dropna().unique())
outcome_dropdown = widgets.Dropdown(
    options=outcome_options,
    description='Outcome:',
    value='All'
)

def update_pie(outcome_filter, region_filter, location_filter, month_filter):
    dff = df_filtered.copy()
    
    if outcome_filter != 'All':
        dff = dff[dff['outcome_group'] == outcome_filter]
    if region_filter != 'All':
        dff = dff[dff['region_group'] == region_filter]
    if location_filter != 'All':
        dff = dff[dff['location_group'] == location_filter]
    if month_filter != 'All':
        dff = dff[dff['month_str'] == month_filter]
    
    crime_counts = dff['crime_group'].value_counts()
    
    if crime_counts.empty:
        fig = go.Figure()
        fig.update_layout(title="No data for selected filters")
        fig.show()
        return
    
    labels = crime_counts.index.tolist()
    values = crime_counts.values.tolist()
    colors = [color_map[label] for label in labels]

    
    fig = go.Figure(data=[go.Pie(
        labels=labels,
        values=values,
        hole=0.4,
        textinfo='percent',
        hovertemplate='%{label}<br>%{percent}',
        marker=dict(colors=colors, line=dict(color='black', width=1.5))
    )])
    
    fig.update_layout(
    width=800,   # width in pixels
    height=500,
    title_text="Crime Group Distribution",
    title_x=0.1,  # Align title a bit to the left (optional)
    legend=dict(
        orientation="v",
        y=0.5,
        yanchor='middle',
        x=1.05,
        xanchor='left'
    ),
    margin=dict(l=50, r=150, t=50, b=50)
)

    
    fig.show()

interactive_plot = widgets.interactive(
    update_pie,
    outcome_filter=outcome_dropdown,
    region_filter=region_dropdown,
    location_filter=location_dropdown,
    month_filter=month_dropdown
)

display(interactive_plot)


interactive(children=(Dropdown(description='Outcome:', options=('All', 'Negative', 'Neutral', 'Positive'), val…

In [5]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Load data
csv_path = "/Users/jamesjackson/Documents/liverpool_crime_analysis/csv_files/clean_data_final.csv"
df = pd.read_csv(csv_path)

# Convert 'month' column to datetime and create a formatted string for dropdown
df['month'] = pd.to_datetime(df['month'])
df['month_str'] = df['month'].dt.strftime('%b %Y')

# Map crime_type to crime_group
group_mapping = {
    'Violence and sexual offences': 'Violent/Sexual',
    'Criminal damage and arson': 'Violent/Sexual',
    'Possession of weapons': 'Weapons Offences',
    'Vehicle crime': 'Vehicle Crime',
    'Burglary': 'Theft/Burglary/Robbery',
    'Theft from the person': 'Theft/Burglary/Robbery',
    'Shoplifting': 'Theft/Burglary/Robbery',
    'Other theft': 'Theft/Burglary/Robbery',
    'Bicycle theft': 'Theft/Burglary/Robbery',
    'Drugs': 'Drug-Related Crime',
    'Anti-social behaviour': 'Anti-Social Behaviour',
    'Other crime': 'Other Crime'
}


df['crime_group'] = df['crime_type'].map(group_mapping)
df = df.dropna(subset=['crime_group'])  # drops any crime_type not mapped above


# Define your top 9 locations to filter for
top_locations = [
    "On or near Parking Area",
    "On or near Supermarket",
    "On or near Shopping Area",
    "On or near Petrol Station",
    "On or near Nightclub",
    "On or near Sports/Recreation Area",
    "On or near Further/Higher Educational Building",
    "On or near Hospital",
    "On or near Police Station"
]

# Create 'location_group' column: top locations or 'Other'
df['location_group'] = df['location'].apply(lambda x: x if x in top_locations else 'Other')

# Extract region from lsoa_name (assumes region is first word before a space)
df['region'] = df['lsoa_name'].apply(lambda x: x.split()[0] if isinstance(x, str) else 'Unknown')

# Find top 5 regions by crime count
top_regions = df['region'].value_counts().nlargest(5).index.tolist()

# Create 'region_group' column: top regions or 'Other'
df['region_group'] = df['region'].apply(lambda x: x if x in top_regions else 'Other')


# Define outcome groups explicitly
positive_outcomes = [
    "Action to be taken by another organisation",
    "Offender given a drugs possession warning",
    "Suspect charged as part of another case",
    "Offender given a caution",
    "Local resolution"
]
neutral_outcomes = [
    "Status update unavailable",
    "Under investigation",
    "Awaiting court outcome",
    "Court result unavailable"
]
negative_outcomes = [
    "Unable to prosecute suspect",
    "Investigation complete; no suspect identified",
    "Further action is not in the public interest",
    "Further investigation is not in the public interest",
    "Formal action is not in the public interest"
]

# Map outcome_group
def map_outcome_group(outcome):
    if outcome in positive_outcomes:
        return 'Positive'
    elif outcome in neutral_outcomes:
        return 'Neutral'
    elif outcome in negative_outcomes:
        return 'Negative'
    else:
        return None

df['outcome_group'] = df['last_outcome_category'].apply(map_outcome_group)

# Drop rows without outcome_group
# df_filtered = df.dropna(subset=['outcome_group'])
df_filtered = df.copy()


# Dropdown options
df['crime_group'] = df['crime_type'].map(group_mapping)
df['crime_group'] = df['crime_group'].fillna('Other Crime')

# Define pastel colors for outcome groups
# Fix keys to match crime_group labels exactly
crime_type_color_map = {
    'Anti-social behaviour': '#A8D5BA',  # Soft Green
    'Bicycle theft': '#C7CEEA',  # Light Lavender
    'Burglary': '#B5EAD7',  # Mint Green
    'Criminal damage and arson': '#A3B4F7',  # Soft Pastel Blue
    'Drugs': '#C0C0C0',  # Pastel Grey
    'Other crime': '#FFDAC1',  # Light Peach
    'Other theft': '#D5A3F7',  # Pastel Purple
    'Possession of weapons': '#E2F0CB',  # Light Lime
    'Shoplifting': '#F7E8A3',  # Pastel Yellow
    'Theft from the person': '#F1CBFF',   # Light Pink
    'Vehicle crime': '#F7C5A3',  # Pastel Orange
    'Violence and sexual offences': '#F7B7A3',  # Soft Coral (pastel red)
}

# Create dropdown widget

month_options = ['All'] + sorted(df['month_str'].unique())
month_dropdown = widgets.Dropdown(
    options=month_options,
    description='Month:',
    value='All'
)

location_options = ['All'] + sorted(df['location_group'].unique())

location_dropdown = widgets.Dropdown(
    options=location_options,
    description='Location:',
    value='All'
)

region_options = ['All'] + sorted(df['region_group'].unique())

region_dropdown = widgets.Dropdown(
    options=region_options,
    description='Region:',
    value='All'
)

outcome_options = ['All'] + sorted(df_filtered['outcome_group'].dropna().unique())
outcome_dropdown = widgets.Dropdown(
    options=outcome_options,
    description='Outcome:',
    value='All'
)
  
def update_crime_type_bar(region_filter, month_filter, outcome_filter, location_filter):
    dff = df_filtered.copy()

    if region_filter != 'All':
        dff = dff[dff['region_group'] == region_filter]
    if month_filter != 'All':
        dff = dff[dff['month_str'] == month_filter]
    if outcome_filter != 'All':
        dff = dff[dff['outcome_group'] == outcome_filter]
    if location_filter != 'All':
        dff = dff[dff['location_group'] == location_filter]

    crime_type_counts = dff['crime_type'].value_counts().sort_values(ascending=False)

    if crime_type_counts.empty:
        fig = go.Figure()
        fig.update_layout(title="No data for selected filters")
        fig.show()
        return

    # Assign colors for each crime_type bar
    colors = [crime_type_color_map.get(crime_type, '#CCCCCC') for crime_type in crime_type_counts.index]

    fig = go.Figure(data=go.Bar(
    x=crime_type_counts.index,
    y=crime_type_counts.values,
    marker=dict(
        color=colors,
        line=dict(color='black', width=1.5)
    )
))

    fig.update_layout(
        title="Distribution of Crime Types",
        xaxis_title="Crime Type",
        yaxis_title="Number of Crimes",
        xaxis_tickangle=-45,
        margin=dict(l=50, r=50, t=50, b=150),
        width=800,   # width in pixels
        height=500,
    )

    fig.show()

interactive_bar = widgets.interactive(
    update_crime_type_bar,
    region_filter=region_dropdown,
    month_filter=month_dropdown,
    outcome_filter=outcome_dropdown,
    location_filter=location_dropdown
)

display(interactive_bar)

interactive(children=(Dropdown(description='Region:', options=('All', 'Knowsley', 'Liverpool', 'Other', 'Sefto…

In [6]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Load data
csv_path = "/Users/jamesjackson/Documents/liverpool_crime_analysis/csv_files/clean_data_final.csv"
df = pd.read_csv(csv_path)

# Convert 'month' column to datetime and create a formatted string for dropdown
df['month'] = pd.to_datetime(df['month'])
df['month_str'] = df['month'].dt.strftime('%b %Y')

# Map crime_type to crime_group
group_mapping = {
    'Violence and sexual offences': 'Violent/Sexual',
    'Criminal damage and arson': 'Violent/Sexual',
    'Possession of weapons': 'Weapons Offences',
    'Vehicle crime': 'Vehicle Crime',
    'Burglary': 'Theft/Burglary/Robbery',
    'Theft from the person': 'Theft/Burglary/Robbery',
    'Shoplifting': 'Theft/Burglary/Robbery',
    'Other theft': 'Theft/Burglary/Robbery',
    'Bicycle theft': 'Theft/Burglary/Robbery',
    'Drugs': 'Drug-Related Crime',
    'Anti-social behaviour': 'Anti-Social Behaviour',
    'Other crime': 'Other Crime'
}


df['crime_group'] = df['crime_type'].map(group_mapping)
df = df.dropna(subset=['crime_group'])  # drops any crime_type not mapped above


# Define your top 9 locations to filter for
top_locations = [
    "On or near Parking Area",
    "On or near Supermarket",
    "On or near Shopping Area",
    "On or near Petrol Station",
    "On or near Nightclub",
    "On or near Sports/Recreation Area",
    "On or near Further/Higher Educational Building",
    "On or near Hospital",
    "On or near Police Station"
]

# Create 'location_group' column: top locations or 'Other'
df['location_group'] = df['location'].apply(lambda x: x if x in top_locations else 'Other')

# Extract region from lsoa_name (assumes region is first word before a space)
df['region'] = df['lsoa_name'].apply(lambda x: x.split()[0] if isinstance(x, str) else 'Unknown')

# Find top 5 regions by crime count
top_regions = df['region'].value_counts().nlargest(5).index.tolist()

# Create 'region_group' column: top regions or 'Other'
df['region_group'] = df['region'].apply(lambda x: x if x in top_regions else 'Other')
df['region_group'] = df['region_group'].replace({'St.': 'St. Helens'})

# Define outcome groups explicitly
positive_outcomes = [
    "Action to be taken by another organisation",
    "Offender given a drugs possession warning",
    "Suspect charged as part of another case",
    "Offender given a caution",
    "Local resolution"
]
neutral_outcomes = [
    "Status update unavailable",
    "Under investigation",
    "Awaiting court outcome",
    "Court result unavailable"
]
negative_outcomes = [
    "Unable to prosecute suspect",
    "Investigation complete; no suspect identified",
    "Further action is not in the public interest",
    "Further investigation is not in the public interest",
    "Formal action is not in the public interest"
]

# Map outcome_group
def map_outcome_group(outcome):
    if outcome in positive_outcomes:
        return 'Positive'
    elif outcome in neutral_outcomes:
        return 'Neutral'
    elif outcome in negative_outcomes:
        return 'Negative'
    else:
        return None

df['outcome_group'] = df['last_outcome_category'].apply(map_outcome_group)

# Drop rows without outcome_group
# df_filtered = df.dropna(subset=['outcome_group'])
df_filtered = df.copy()


# Dropdown options
df['crime_group'] = df['crime_type'].map(group_mapping)
df['crime_group'] = df['crime_group'].fillna('Other Crime')

# Define pastel colors for outcome groups
# Fix keys to match crime_group labels exactly
crime_type_color_map = {
    'Anti-social behaviour': '#C0C0C0',          # Pastel Grey
    'Bicycle theft': '#A8D5BA',                   # Soft Green
    'Burglary': '#A8D5BA',                        # Soft Green (same as Bicycle theft)
    'Criminal damage and arson': '#F7B7A3',      # Soft Coral (pastel red)
    'Drugs': '#A3B4F7',                           # Soft Pastel Blue
    'Other crime': '#F7C5A3',                     # Pastel Orange
    'Other theft': '#A8D5BA',                      # Soft Green (same as Burglary)
    'Possession of weapons': '#D5A3F7',           # Pastel Purple
    'Shoplifting': '#A8D5BA',                      # Soft Green (same as Burglary)
    'Theft from the person': '#A8D5BA',           # Soft Green (same as Burglary)
    'Vehicle crime': '#F7E8A3',                    # Pastel Yellow
    'Violence and sexual offences': '#F7B7A3'     # Soft Coral (pastel red)
}

# Create dropdown widget

month_options = ['All'] + sorted(df['month_str'].unique())
month_dropdown = widgets.Dropdown(
    options=month_options,
    description='Month:',
    value='All'
)

crime_group_options = ['All'] + sorted(df['crime_group'].dropna().unique())

crime_group_dropdown = widgets.Dropdown(
    options=crime_group_options,
    description='Crime Group:',
    value='All'
)

location_options = ['All'] + sorted(df['location_group'].unique())

location_dropdown = widgets.Dropdown(
    options=location_options,
    description='Location:',
    value='All'
)

region_options = ['All'] + sorted(df['region_group'].unique())

region_dropdown = widgets.Dropdown(
    options=region_options,
    description='Region:',
    value='All'
)

outcome_options = ['All'] + sorted(df_filtered['outcome_group'].dropna().unique())
outcome_dropdown = widgets.Dropdown(
    options=outcome_options,
    description='Outcome:',
    value='All'
)
  
def update_crime_volume_region(region_filter, month_filter, outcome_filter, location_filter, crime_group_filter):
    dff = df_filtered.copy()

    # Apply filters
    if month_filter != 'All':
        dff = dff[dff['month_str'] == month_filter]
    if outcome_filter != 'All':
        dff = dff[dff['outcome_group'] == outcome_filter]
    if location_filter != 'All':
        dff = dff[dff['location_group'] == location_filter]
    if crime_group_filter != 'All':
        dff = dff[dff['crime_group'] == crime_group_filter]

    # Exclude 'Other' from top regions
    dff = dff[dff['region_group'] != 'Other']

    # Group and plot
    region_counts = dff['region_group'].value_counts().sort_values(ascending=False)

    if region_counts.empty:
        fig = go.Figure()
        fig.update_layout(title="No data for selected filters")
        fig.show()
        return

    pastel_colors = ['#F7B7A3', '#A8D5BA', '#A3B4F7', '#F7E8A3', '#D5A3F7']

    fig = go.Figure(data=go.Bar(
        x=region_counts.index,
        y=region_counts.values,
        marker=dict(
            color=pastel_colors[:len(region_counts)],
            line=dict(color='#333333', width=1.5)
        )
    ))

    fig.update_layout(
        width=800,   # width in pixels
        height=500,
        title="Crime Volume by Region",
        xaxis_title="Region",
        yaxis_title="Number of Crimes",
        margin=dict(l=50, r=50, t=50, b=50),
    )

    fig.show()

 

interactive_region_bar = widgets.interactive(
    update_crime_volume_region,
    region_filter=widgets.fixed('All'),
    month_filter=month_dropdown,
    outcome_filter=outcome_dropdown,
    location_filter=location_dropdown,
    crime_group_filter=crime_group_dropdown
)

display(interactive_region_bar)


interactive(children=(Dropdown(description='Month:', options=('All', 'Apr 2025', 'Aug 2024', 'Dec 2024', 'Feb …

In [7]:
import plotly.graph_objects as go
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Load data
csv_path = "/Users/jamesjackson/Documents/liverpool_crime_analysis/csv_files/clean_data_final.csv"
df = pd.read_csv(csv_path, parse_dates=['month'])

# Drop rows with critical missing values early
df.dropna(subset=['month', 'crime_type', 'lsoa_name'], inplace=True)

# Add month_str for filtering and display
df['month_str'] = df['month'].dt.strftime('%b %Y')

# Map crime_type to crime_group
group_mapping = {
    'Violence and sexual offences': 'Violent / Sexual',
    'Criminal damage and arson': 'Violent / Sexual',
    'Vehicle crime': 'Vehicle Crime',
    'Burglary': 'Theft / Burglary / Robbery',
    'Theft from the person': 'Theft / Burglary / Robbery',
    'Shoplifting': 'Theft / Burglary / Robbery',
    'Other theft': 'Theft / Burglary / Robbery',
    'Bicycle theft': 'Theft / Burglary / Robbery',
    'Drugs': 'Drug-Related Crime',
    'Anti-social behaviour': 'Anti-Social Behaviour',
    'Other crime': 'Other Crime',
    'Possession of weapons': 'Weapons Offences'
}

df['crime_group'] = df['crime_type'].map(group_mapping).fillna('Other')

# Identify top 9 known locations
top_locations = [
    "On or near Parking Area",
    "On or near Supermarket",
    "On or near Shopping Area",
    "On or near Petrol Station",
    "On or near Nightclub",
    "On or near Sports/Recreation Area",
    "On or near Further/Higher Educational Building",
    "On or near Hospital",
    "On or near Police Station"
]
df['location_group'] = df['location'].apply(lambda x: x if x in top_locations else 'Other')

# Extract region and determine top 5
df['region'] = df['lsoa_name'].apply(lambda x: x.split()[0] if isinstance(x, str) else 'Unknown')
top_regions = df['region'].value_counts().nlargest(5).index.tolist()
df['region_group'] = df['region'].apply(lambda x: x if x in top_regions else 'Other')

# Outcome group mapping
positive_outcomes = [
    "Action to be taken by another organisation",
    "Offender given a drugs possession warning",
    "Suspect charged as part of another case",
    "Offender given a caution",
    "Local resolution"
]
neutral_outcomes = [
    "Status update unavailable",
    "Under investigation",
    "Awaiting court outcome",
    "Court result unavailable"
]
negative_outcomes = [
    "Unable to prosecute suspect",
    "Investigation complete; no suspect identified",
    "Further action is not in the public interest",
    "Further investigation is not in the public interest",
    "Formal action is not in the public interest"
]

def map_outcome_group(outcome):
    if outcome in positive_outcomes:
        return 'Positive'
    elif outcome in neutral_outcomes:
        return 'Neutral'
    elif outcome in negative_outcomes:
        return 'Negative'
    else:
        return None

df['outcome_group'] = df['last_outcome_category'].apply(map_outcome_group)

# Final filtered copy
df_filtered = df.copy()

# Dropdowns
crime_group_options = ['All'] + sorted(set(group_mapping.values()))
crime_group_dropdown = widgets.Dropdown(
    options=crime_group_options,
    description='Crime Group:',
    value='All'
)

location_options = ['All'] + sorted(df['location_group'].unique())
location_dropdown = widgets.Dropdown(
    options=location_options,
    description='Location:',
    value='All'
)

region_options = ['All'] + sorted(df['region_group'].unique())
region_dropdown = widgets.Dropdown(
    options=region_options,
    description='Region:',
    value='All'
)

outcome_options = ['All', 'Positive', 'Neutral', 'Negative']
outcome_dropdown = widgets.Dropdown(
    options=outcome_options,
    description='Outcome:',
    value='All'
)

def update_monthly_trend_dual_axis(crime_group_filter, region_filter, location_filter, outcome_filter):
    dff = df_filtered.copy()
    
    # Apply filters
    if crime_group_filter != 'All':
        dff = dff[dff['crime_group'] == crime_group_filter]
    if region_filter != 'All':
        dff = dff[dff['region_group'] == region_filter]
    if location_filter != 'All':
        dff = dff[dff['location_group'] == location_filter]
    if outcome_filter != 'All':
        dff = dff[dff['outcome_group'] == outcome_filter]

    # Total crimes (unfiltered)
    total_counts = df_filtered.groupby('month').size().sort_index()

    # Filtered crimes count by month (aligned to total_counts index)
    filtered_counts = dff.groupby('month').size().reindex(total_counts.index, fill_value=0)

    # Month labels for x-axis
    months_str = total_counts.index.strftime('%b %Y')

    # Pastel colors by month number (Jan-Dec)
    pastel_colors = [
        '#D5A3F7',   
        '#E2F0CB',  
        '#F7E8A3', 
        '#F1CBFF',  
        '#F7C5A3',  
        '#F7B7A3',  
        '#A8D5BA',  
        '#C7CEEA',  
        '#B5EAD7',  
        '#A3B4F7',  
        '#C0C0C0',  
        '#FFDAC1',  
    ]

    # Map each month to its pastel color by month number
    bar_colors = [pastel_colors[m.month - 1] for m in total_counts.index]

    fig = go.Figure()

    # Bars for total crimes with pastel colors
    fig.add_trace(go.Bar(
        x=months_str,
        y=total_counts.values,
        marker_color=bar_colors,
        marker_line_color='black',  # Outline color
        marker_line_width=1.5,
        name='Total Crimes',
        yaxis='y1',
        opacity=0.7
    ))

    # Line for filtered crimes in police blue
    fig.add_trace(go.Scatter(
        x=months_str,
        y=filtered_counts.values,
        mode='lines+markers',
        line=dict(color='#1F77B4', width=3),  # Police blue
        name=f'{crime_group_filter} Trend',
        yaxis='y2'
    ))

    total_crimes = total_counts.values
    filtered_crimes = filtered_counts.values

    max_value = max(total_crimes) * 1.1  # a bit more than max total crimes for padding
    max_value_filtered = max(filtered_crimes) * 1.1  # a bit more than max filtered crimes

    fig.update_layout(
        width=800,   # width in pixels
        height=500,
        showlegend=False,
        title={
            'text': "Trends in Total and Filtered Crime Counts Over Time",
            'font': {'size': 16}  # smaller size, default is around 20-24
        },
        yaxis=dict(
            zeroline=True,
            zerolinecolor='black',
            zerolinewidth=2,
            range=[0, max_value],
            showline=True,
            linecolor='black',
            mirror=True,
            anchor='x',
            ),
        yaxis2=dict(
            zeroline=True,
            zerolinecolor='black',
            zerolinewidth=2,
            range=[0, max_value_filtered],
            overlaying='y',
            side='right',
            showline=True,
            linecolor='black',
            mirror=True
        ),
        xaxis=dict(
            zeroline=True,
            zerolinecolor='black',
            zerolinewidth=2,
            showline=True,
            linecolor='black',
            mirror=True,
            anchor='y'
        )
    )

    fig.show()

interactive_monthly_trend = widgets.interactive(
    update_monthly_trend_dual_axis,
    crime_group_filter=crime_group_dropdown,
    region_filter=region_dropdown,
    location_filter=location_dropdown,
    outcome_filter=outcome_dropdown
)

display(interactive_monthly_trend)


interactive(children=(Dropdown(description='Crime Group:', options=('All', 'Anti-Social Behaviour', 'Drug-Rela…

In [8]:
import plotly.graph_objects as go
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from matplotlib import colors as mcolors
import numpy as np

# Load data
csv_path = "/Users/jamesjackson/Documents/liverpool_crime_analysis/csv_files/clean_data_final.csv"
df = pd.read_csv(csv_path, parse_dates=['month'])

# Drop rows with critical missing values early
df.dropna(subset=['month', 'crime_type', 'lsoa_name'], inplace=True)

# Add month_str for filtering and display
df['month_str'] = df['month'].dt.strftime('%b %Y')

# Map crime_type to crime_group
group_mapping = {
    'Violence and sexual offences': 'Violent / Sexual',
    'Criminal damage and arson': 'Violent / Sexual',
    'Vehicle crime': 'Vehicle Crime',
    'Burglary': 'Theft / Burglary / Robbery',
    'Theft from the person': 'Theft / Burglary / Robbery',
    'Shoplifting': 'Theft / Burglary / Robbery',
    'Other theft': 'Theft / Burglary / Robbery',
    'Bicycle theft': 'Theft / Burglary / Robbery',
    'Drugs': 'Drug-Related Crime',
    'Anti-social behaviour': 'Anti-Social Behaviour',
    'Other crime': 'Other Crime',
    'Possession of weapons': 'Weapons Offences'
}
df['crime_group'] = df['crime_type'].map(group_mapping).fillna('Other')

# Identify top 9 known locations
top_locations = [
    "On or near Parking Area",
    "On or near Supermarket",
    "On or near Shopping Area",
    "On or near Petrol Station",
    "On or near Nightclub",
    "On or near Sports/Recreation Area",
    "On or near Further/Higher Educational Building",
    "On or near Hospital",
    "On or near Police Station"
]
df['location_group'] = df['location'].apply(lambda x: x if x in top_locations else 'Other')

# Extract region and determine top 5
df['region'] = df['lsoa_name'].apply(lambda x: x.split()[0] if isinstance(x, str) else 'Unknown')
top_regions = df['region'].value_counts().nlargest(5).index.tolist()
df['region_group'] = df['region'].apply(lambda x: x if x in top_regions else None)  # Remove "Other"

# Outcome group mapping
positive_outcomes = [
    "Action to be taken by another organisation",
    "Offender given a drugs possession warning",
    "Suspect charged as part of another case",
    "Offender given a caution",
    "Local resolution"
]
neutral_outcomes = [
    "Status update unavailable",
    "Under investigation",
    "Awaiting court outcome",
    "Court result unavailable"
]
negative_outcomes = [
    "Unable to prosecute suspect",
    "Investigation complete; no suspect identified",
    "Further action is not in the public interest",
    "Further investigation is not in the public interest",
    "Formal action is not in the public interest"
]

def map_outcome_group(outcome):
    if outcome in positive_outcomes:
        return 'Positive'
    elif outcome in neutral_outcomes:
        return 'Neutral'
    elif outcome in negative_outcomes:
        return 'Negative'
    else:
        return None

df['outcome_group'] = df['last_outcome_category'].apply(map_outcome_group)

# Final filtered copy
df_filtered = df[df['region_group'].notna()].copy()

# Dropdowns
crime_group_options = ['All'] + sorted(set(group_mapping.values()))
crime_group_dropdown = widgets.Dropdown(
    options=crime_group_options,
    description='Crime Group:',
    value='All'
)

location_options = ['All'] + sorted(df['location_group'].unique())
location_dropdown = widgets.Dropdown(
    options=location_options,
    description='Location:',
    value='All'
)

month_options = ['All'] + sorted(df_filtered['month_str'].unique(), key=lambda x: pd.to_datetime(x, format='%b %Y'))
month_dropdown = widgets.Dropdown(
    options=month_options,
    description='Month:',
    value='All'
)

def hex_to_rgb(hex_color):
    return np.array(mcolors.to_rgb(hex_color))

def rgb_to_hex(rgb):
    return mcolors.to_hex(rgb)

def get_gradient_color(value, min_val, max_val, low_hex, mid_hex, high_hex):
    low_rgb = hex_to_rgb(low_hex)
    mid_rgb = hex_to_rgb(mid_hex)
    high_rgb = hex_to_rgb(high_hex)
    ratio = (value - min_val) / (max_val - min_val) if max_val > min_val else 0.5

    if ratio < 0.5:
        # Interpolate from low to mid
        interp = low_rgb + (mid_rgb - low_rgb) * (ratio / 0.5)
    else:
        # Interpolate from mid to high
        interp = mid_rgb + (high_rgb - mid_rgb) * ((ratio - 0.5) / 0.5)
    return rgb_to_hex(interp)

def update_positive_outcome_by_region(month_filter, crime_group_filter, location_filter):
    dff = df_filtered.copy()

    if month_filter != 'All':
        dff = dff[dff['month_str'] == month_filter]
    if crime_group_filter != 'All':
        dff = dff[dff['crime_group'] == crime_group_filter]
    if location_filter != 'All':
        dff = dff[dff['location_group'] == location_filter]

    grouped = dff.groupby('region_group').agg(
        total_crimes=('outcome_group', 'size'),
        positive_outcomes=('outcome_group', lambda x: (x == 'Positive').sum())
    )
    grouped['positive_rate'] = (grouped['positive_outcomes'] / grouped['total_crimes']) * 100
    grouped = grouped.sort_values('positive_rate', ascending=False)

    min_val, max_val = grouped['positive_rate'].min(), grouped['positive_rate'].max()

    # Apply gradient coloring
    bar_colors = [
        get_gradient_color(v, min_val, max_val, '#F7B7A3', '#FFFFFF', '#A8D5BA')
        for v in grouped['positive_rate']
    ]

    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=grouped['positive_rate'],
        y=grouped.index,
        orientation='h',
        text=grouped['positive_rate'].round(1).astype(str) + '%',
        textposition='auto',
        marker_color=bar_colors,
        marker_line_color='black',  # Outline color
        marker_line_width=1.5  
    ))

    fig.update_layout(
        title={
            'text': 'Positive Outcome Rates by Region',
            'font': {'size': 18}
        },
        xaxis=dict(
            title='Positive Outcome Rate (%)',
            range=[0, grouped['positive_rate'].max() * 1.1]
        ),
        yaxis=dict(
            title='Region',
            autorange='reversed'
        ),
        margin=dict(l=100, r=20, t=60, b=50),
        height=400,
        template='plotly_white'
    )

    fig.show()

interactive_positive_outcome = widgets.interactive(
    update_positive_outcome_by_region,
    month_filter=month_dropdown,
    crime_group_filter=crime_group_dropdown,
    location_filter=location_dropdown
)

display(interactive_positive_outcome)


interactive(children=(Dropdown(description='Month:', options=('All', 'Jun 2024', 'Jul 2024', 'Aug 2024', 'Sep …

In [9]:
import plotly.graph_objects as go
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Load your data
csv_path = "/Users/jamesjackson/Documents/liverpool_crime_analysis/csv_files/clean_data_final.csv"
df = pd.read_csv(csv_path, parse_dates=['month'])

# Prepare month string for filtering
df['month_str'] = df['month'].dt.strftime('%b %Y')

# Crime group mapping (manual)
group_mapping = {
    'Violence and sexual offences': 'Violent / Sexual',
    'Criminal damage and arson': 'Violent / Sexual',  # optional inclusion
    'Vehicle crime': 'Theft / Burglary / Robbery',
    'Burglary': 'Theft / Burglary / Robbery',
    'Theft from the person': 'Theft / Burglary / Robbery',
    'Shoplifting': 'Theft / Burglary / Robbery',
    'Other theft': 'Theft / Burglary / Robbery',
    'Bicycle theft': 'Theft / Burglary / Robbery',
    'Drugs': 'Drug-Related Crime',
    'Anti-social behaviour': 'Anti-Social Behaviour',
    'Other crime': 'Other',
    'Possession of weapons': 'Drug-Related Crime'  # adjust if needed
}
df['crime_group'] = df['crime_type'].map(group_mapping).fillna('Other')

# Outcome groups mapping
positive_outcomes = [
    "Action to be taken by another organisation",
    "Offender given a drugs possession warning",
    "Suspect charged as part of another case",
    "Offender given a caution",
    "Local resolution"
]
neutral_outcomes = [
    "Status update unavailable",
    "Under investigation",
    "Awaiting court outcome",
    "Court result unavailable"
]
negative_outcomes = [
    "Unable to prosecute suspect",
    "Investigation complete; no suspect identified",
    "Further action is not in the public interest",
    "Further investigation is not in the public interest",
    "Formal action is not in the public interest"
]

def map_outcome_group(outcome):
    if outcome in positive_outcomes:
        return 'Positive'
    elif outcome in neutral_outcomes:
        return 'Neutral'
    elif outcome in negative_outcomes:
        return 'Negative'
    else:
        return None

df['outcome_group'] = df['last_outcome_category'].apply(map_outcome_group)

# Top 9 locations (cleaned to remove "On or near ")
top_locations_raw = [
    "On or near Parking Area",
    "On or near Supermarket",
    "On or near Shopping Area",
    "On or near Petrol Station",
    "On or near Nightclub",
    "On or near Sports/Recreation Area",
    "On or near Further/Higher Educational Building",
    "On or near Hospital",
    "On or near Police Station"
]

def clean_location(loc):
    if isinstance(loc, str) and loc in top_locations_raw:
        return loc.replace("On or near ", "")
    else:
        return 'Other'

df['location_group'] = df['location'].apply(clean_location)

# Extract region from lsoa_name (first word)
df['region'] = df['lsoa_name'].apply(lambda x: x.split()[0] if isinstance(x, str) else 'Unknown')

# Dropdown options for filters
month_options = ['All'] + sorted(df['month_str'].unique(), key=lambda x: pd.to_datetime(x, format='%b %Y'))
region_options = ['All'] + sorted(df['region'].unique())
outcome_options = ['All', 'Positive', 'Neutral', 'Negative']

month_dropdown = widgets.Dropdown(options=month_options, description='Month:', value='All')
region_dropdown = widgets.Dropdown(options=region_options, description='Region:', value='All')
outcome_dropdown = widgets.Dropdown(options=outcome_options, description='Outcome Group:', value='All')

# Define colors for the four crime groups
color_map = {
    'Violent / Sexual': '#F7B7A3',        # Soft Coral (pastel red)
    'Theft / Burglary / Robbery': '#A8D5BA',  # Soft Green
    'Anti-Social Behaviour': '#C0C0C0',   # Pastel Grey
    'Drug-Related Crime': '#A3B4F7',      # Soft Pastel Blue
}

def plot_grouped_bar(selected_month, selected_region, selected_outcome):
    dff = df.copy()

    # Apply filters
    if selected_month != 'All':
        dff = dff[dff['month_str'] == selected_month]
    if selected_region != 'All':
        dff = dff[dff['region'] == selected_region]
    if selected_outcome != 'All':
        dff = dff[dff['outcome_group'] == selected_outcome]

    # Filter to top 9 locations only
    dff = dff[dff['location_group'] != 'Other']

    # Crime groups to include
    crime_groups_to_include = [
        'Violent / Sexual',
        'Theft / Burglary / Robbery',
        'Anti-Social Behaviour',
        'Drug-Related Crime'
    ]

    dff = dff[dff['crime_group'].isin(crime_groups_to_include)]

    # Group and count by location and crime group
    grouped = dff.groupby(['location_group', 'crime_group']).size().reset_index(name='count')

    # Pivot for grouped bar chart
    pivot_df = grouped.pivot(index='location_group', columns='crime_group', values='count').fillna(0)

    # Make sure all crime groups are present as columns (fill 0 if missing)
    pivot_df = pivot_df.reindex(columns=crime_groups_to_include, fill_value=0)

    # Sort locations by total crime count descending
    pivot_df['total'] = pivot_df.sum(axis=1)
    pivot_df = pivot_df.sort_values('total', ascending=False)
    pivot_df.drop(columns='total', inplace=True)

    # Create figure
    fig = go.Figure()

    for crime_group in crime_groups_to_include:
        fig.add_trace(go.Bar(
            name=crime_group,
            x=pivot_df.index,
            y=pivot_df[crime_group],
            marker=dict(color=color_map[crime_group], line=dict(color='black', width=1))
        ))

    fig.update_layout(
        barmode='group',
        title="Crime Counts by Location and Crime Group",
        xaxis_title="Location",
        yaxis_title="Number of Crimes",
        height=600,
        width=900,
        template='plotly_white'
    )

    fig.show()

widgets.interact(
    plot_grouped_bar,
    selected_month=month_dropdown,
    selected_region=region_dropdown,
    selected_outcome=outcome_dropdown
)


interactive(children=(Dropdown(description='Month:', options=('All', 'Jun 2024', 'Jul 2024', 'Aug 2024', 'Sep …

<function __main__.plot_grouped_bar(selected_month, selected_region, selected_outcome)>

In [10]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [11]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import LinearSegmentedColormap

# Place names and coordinates (latitude, longitude), without Widnes and Runcorn
places = {
    'Liverpool': (53.4084, -2.9916),
    'West Kirby': (53.3859, -3.1808),
    'Birkenhead': (53.3933, -3.0136),
    'Wallasey': (53.4161, -3.0606),
    'Heswall': (53.3506, -3.1001),
    'Bromborough': (53.3425, -3.0082),
    'Halewood': (53.3743, -2.8311),
    'Prescot': (53.4394, -2.8234),
    'St Helens': (53.4543, -2.7443),
    'Bootle': (53.4584, -3.0121),
    'Litherland': (53.4590, -3.0208),
    'Crosby': (53.4797, -3.0407),
    'Kirkby': (53.4857, -2.8642),
    'Maghull': (53.5227, -2.9406),
    'Formby': (53.5307, -3.0562),
    'Southport': (53.6453, -3.0081)
}

# Custom colormap white → yellow → orange → red
custom_cmap = LinearSegmentedColormap.from_list(
    'custom_heatmap',
    ['#ffffff', '#ffff00', '#ffa500', '#ff0000']  # white, yellow, orange, red
)

plt.figure(figsize=(12, 10))
plt.hexbin(
    gdf['longitude'], gdf['latitude'],
    gridsize=50,
    cmap=custom_cmap,
    norm=colors.LogNorm(),  # Logarithmic scale
    mincnt=1
)
plt.colorbar(label='Number of Crimes (log scale)')
plt.title('Crime Density Hexbin Plot - Liverpool (Log Scale)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

for place, (lat, lon) in places.items():
    plt.plot(lon, lat, 'ko', markersize=5)  # black dot
    plt.text(lon, lat + 0.005, place, fontsize=9, fontweight='bold', color='black',
             ha='center', va='bottom', alpha=0.7,
             bbox=dict(facecolor='white', alpha=0.6, edgecolor='none', boxstyle='round,pad=0.3'))
plt.tight_layout()
plt.savefig('/Users/jamesjackson/Documents/liverpool_crime_analysis/crime_density_hexbin_plot.png')
plt.show()


NameError: name 'gdf' is not defined

<Figure size 1200x1000 with 0 Axes>

In [12]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
from ipywidgets import interact, widgets
from IPython.display import display

# --- Load Data ---
# Replace this with your actual DataFrame loading method
# df = pd.read_csv('your_crime_data.csv')

# Drop missing coordinates
df = df.dropna(subset=['latitude', 'longitude'])

# --- Clean LSOA Names for Region/Location Filters ---
df['lsoa_name_clean'] = df['lsoa_name'].str.replace('On or near ', '', case=False).str.strip()

# --- Extract Month and Year ---
df['month'] = pd.to_datetime(df['month'])
df['month_str'] = df['month'].dt.strftime('%Y-%m')

# --- Define Region Mapping (adjust as needed) ---
def extract_region(name):
    # Naive rule-based region extraction from LSOA name
    for region in ['Liverpool', 'Wirral', 'Sefton', 'Knowsley', 'St Helens']:
        if region.lower() in name.lower():
            return region
    return 'Other'

df['region'] = df['lsoa_name_clean'].apply(extract_region)

# --- Crime & Outcome Group Mapping ---
crime_group_map = {
    'Violence and sexual offences': 'Violent / Sexual',
    'Other theft': 'Theft / Burglary / Robbery',
    'Burglary': 'Theft / Burglary / Robbery',
    'Robbery': 'Theft / Burglary / Robbery',
    'Shoplifting': 'Theft / Burglary / Robbery',
    'Theft from the person': 'Theft / Burglary / Robbery',
    'Vehicle crime': 'Theft / Burglary / Robbery',
    'Bicycle theft': 'Theft / Burglary / Robbery',
    'Anti-social behaviour': 'Anti-Social Behaviour',
    'Drugs': 'Drug-Related Crime',
}
df['crime_group'] = df['crime_type'].map(crime_group_map)
df['crime_group'] = df['crime_group'].fillna('Other')

outcome_group_map = {
    'Investigation complete; no suspect identified': 'No Suspect Identified',
    'Offender given a caution': 'Caution Issued',
    'Offender given a drugs possession warning': 'Caution Issued',
    'Offender given penalty notice': 'Caution Issued',
    'Offender sent to prison': 'Positive Outcome',
    'Offender fined': 'Positive Outcome',
    'Unable to prosecute suspect': 'No Further Action',
    'Suspect charged': 'Positive Outcome',
    'Awaiting court outcome': 'Court Pending',
    'Further investigation is not in the public interest': 'No Further Action',
}
df['outcome_group'] = df['last_outcome_category'].map(outcome_group_map)
df['outcome_group'] = df['outcome_group'].fillna('Unknown')

# --- Places for Annotation ---
places = {
    'Liverpool': (53.4084, -2.9916),
    'West Kirby': (53.3859, -3.1808),
    'Birkenhead': (53.3933, -3.0136),
    'Wallasey': (53.4161, -3.0606),
    'Heswall': (53.3506, -3.1001),
    'Bromborough': (53.3425, -3.0082),
    'Halewood': (53.3743, -2.8311),
    'Prescot': (53.4394, -2.8234),
    'St Helens': (53.4543, -2.7443),
    'Bootle': (53.4584, -3.0121),
    'Litherland': (53.4590, -3.0208),
    'Crosby': (53.4797, -3.0407),
    'Kirkby': (53.4857, -2.8642),
    'Maghull': (53.5227, -2.9406),
    'Formby': (53.5307, -3.0562),
    'Southport': (53.6453, -3.0081)
}

# --- Custom Colormap ---
custom_cmap = LinearSegmentedColormap.from_list(
    'custom_heatmap', ['#ffffff', '#ffff00', '#ffa500', '#ff0000']
)

# --- Interactive Plotting Function ---
def plot_hexbin(region, month, location, crime_group, outcome_group):
    dff = df.copy()

    if region != 'All':
        dff = dff[dff['region'] == region]
    if month != 'All':
        dff = dff[dff['month_str'] == month]
    if location != 'All':
        dff = dff[dff['lsoa_name_clean'] == location]
    if crime_group != 'All':
        dff = dff[dff['crime_group'] == crime_group]
    if outcome_group != 'All':
        dff = dff[dff['outcome_group'] == outcome_group]

    if dff.empty:
        print("No data matches the selected filters.")
        return

    plt.figure(figsize=(12, 10))
    hb = plt.hexbin(
        dff['longitude'], dff['latitude'],
        gridsize=50,
        cmap=custom_cmap,
        norm=mcolors.LogNorm(),
        mincnt=1
    )
    plt.colorbar(label='Number of Crimes (log scale)')
    plt.title(f"Crime Density Hexbin Plot\nRegion: {region}, Month: {month}, Location: {location}\nCrime: {crime_group}, Outcome: {outcome_group}")
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    for place, (lat, lon) in places.items():
        plt.plot(lon, lat, 'ko', markersize=5)
        plt.text(lon, lat + 0.005, place, fontsize=9, fontweight='bold', color='black',
                 ha='center', va='bottom', alpha=0.7,
                 bbox=dict(facecolor='white', alpha=0.6, edgecolor='none', boxstyle='round,pad=0.3'))

    plt.tight_layout()
    plt.show()

# --- Widgets ---
region_options = ['All'] + sorted(df['region'].dropna().unique())
month_options = ['All'] + sorted(df['month_str'].dropna().unique())
location_options = ['All'] + sorted(df['lsoa_name_clean'].dropna().unique())
crime_group_options = ['All'] + sorted(df['crime_group'].unique())
outcome_group_options = ['All'] + sorted(df['outcome_group'].unique())

interact(
    plot_hexbin,
    region=widgets.Dropdown(options=region_options, description='Region'),
    month=widgets.Dropdown(options=month_options, description='Month'),
    location=widgets.Dropdown(options=location_options, description='Location'),
    crime_group=widgets.Dropdown(options=crime_group_options, description='Crime Group'),
    outcome_group=widgets.Dropdown(options=outcome_group_options, description='Outcome Group')
);


interactive(children=(Dropdown(description='Region', options=('All', 'Knowsley', 'Liverpool', 'Other', 'Sefton…

In [ ]:
# 4. Crime Type Distibution Filters

crime_type_filters = {}

# 5. Regional Crime Volume Filters

regional_crime_filters = {}

# 6. Crime Over Time Filters

crime_over_time_filters = {}

# 7. Positive Outcome Rates Filters

positive_outcome_rates_filters = {}
    
    
# 8. Crime Group by Location Filters

crime_group_by_location_filters = {}

# 9. Merseyside Crime Hexbin Filters

crime_hexbin_filters = {}